# Import

In [1]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import joblib
import json
import cv2
from time import time
import threading
import math
import pickle
from glob import glob
from time import time

# coding=utf-8
from __future__ import absolute_import, print_function
import torch
from sklearn import metrics

from DataSet.dataset import get_iwildcam_loader, data_prefetcher
from Utils.train_utils import cross_entropy,focal_loss, get_optimizer
from Utils.train_utils import mixup_data, mixup_criterion
from Models.model_factory import create_model

import warnings
warnings.filterwarnings("ignore")

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
%load_ext nb_black

print('device:', device)

device: cpu


<IPython.core.display.Javascript object>

# Prepare data

1. rewrite_train_data_json => đổi anotation dạng json sang csv (CTT và iN) => `data/train_file.csv`, `data/test_file.csv`
2. `data/train_file.csv` sau khi  => dev location = 46, train location != 4at6, lấy các cột date_captured, category_id, file_name (đổi đường dẫn cho đúng), rights_holder, id, width, height, location (k có thì -1) => `train_file.csv`, `dev_file.csv`
3. `data/test_file.csv` lấy các cột category_id, file_name => `test_file.csv`

In [2]:
DATASET={'CCT':'iWildCam_2019_CCT','iNat':'iWildCam_2019_iNat_Idaho','IDFG':'iWildCam_IDFG'} #_images_small
DATA_DIR='./data/'
ANNOTATION_DIR =DATA_DIR+ 'iWildCam_2019_Annotations/'

<IPython.core.display.Javascript object>

In [4]:
def rewrite_train_data_json(dataset='CCT'):
	json_path=ANNOTATION_DIR+DATASET[dataset]+'.json'

	json_data = json.load(open(json_path,'r'))
	images = json_data['images']
	annotations = json_data['annotations']
	csv_data={'category_id':[],'date_captured':[],'id':[],'file_name':[],
	          'rights_holder':[],'width':[],'height':[],'location':[]}
	print('len of  data:',dataset,len(images))
	for ii,(img, annot) in enumerate(zip(images,annotations)):
		if img['id'] != annot['image_id']:
			print('there are some error in',ii,img['id'],annot['image_id'])
		if 'date_captured' in img:
			date=img['date_captured']
		elif 'datetime' in img:
			date = img['datetime']
		else:
			date = json_data['info']['date_created']
		csv_data['date_captured'] += [date]
		csv_data['category_id'] += [annot['category_id']]
		csv_data['file_name'] += [img['file_name']]
		csv_data['rights_holder'] += [img['rights_holder']]
		csv_data['id'] += [img['id']]
		csv_data['width'] += [img['width']]
		csv_data['height'] += [img['height']]
		if 'location' in img:
			locat = img['location']
		else:
			locat=-1
		csv_data['location'] += [locat]


	csv_data = pd.DataFrame(csv_data)
	csv_data.to_csv(ANNOTATION_DIR+DATASET[dataset]+'.csv',index=False)


def split_train_dev(CCT=True,iNat=True):
	columns=['category_id','date_captured','id','file_name',
	          'rights_holder','width','height','location']
	train=pd.DataFrame()
	if CCT:
		temp=pd.read_csv(ANNOTATION_DIR+DATASET['CCT']+'.csv')[columns]
		temp['dataset'] = 'CCT'
		temp['file_name'] = temp['file_name'].map(lambda x:'iWildCam_2019_CCT_images_small/'+x)
		print('use CCT data',temp.shape)
		train=pd.concat([train,temp])

	if iNat:
		temp=pd.read_csv(ANNOTATION_DIR+DATASET['iNat']+'.csv')[columns]
		temp['dataset'] = 'iNat'
		temp['file_name'] = temp['file_name'].map(lambda x: 'iWildCam_2019_iNat_Idaho/' + x)
		print('use iNat data',temp.shape)
		train=pd.concat([train,temp])


	print('train shape',train.shape)
	#train=train.sample(frac=1,random_state=0).reset_index(drop=True)

	dev_file = train[train['location'] == 46]  # 46
	train_file = train[train['location'] != 46]


	train_file.to_csv(DATA_DIR+'train_file.csv',index=False)
	dev_file.to_csv(DATA_DIR+'dev_file.csv',index=False)

	print('category ratio for train data:')
	cnt = Counter(train_file['category_id'].values)
	L = len(train_file)
	for ii in range(23):
		print(ii, cnt[ii], cnt[ii] / L)

	print('category ratio for dev data:')
	cnt = Counter(dev_file['category_id'].values)
	L = len(dev_file)
	for ii in range(23):
		print(ii, cnt[ii], cnt[ii] / L)


def save_test():
	columns=['date_captured','id','file_name',
	          'rights_holder','width','height','location']
	test = pd.read_csv(DATA_DIR+'test.csv')[columns]
	test['dataset'] = 'test'
	test['category_id'] = -1
	test['file_name'] = test['file_name'].map(lambda x:'test_images/'+x)
	print('test shape',test.shape) #153730

	test.to_csv(DATA_DIR+'test_file.csv',index=False)

full_data_dir='data/raw_data/iWildCam_2019_IDFG/iWildCam_IDFG_images/'
def get_test_orig_size_split(test_file,name=0):
	name=str(name)
	print('get_test_orig_size_split for thread',name,test_file.shape)
	file_names= test_file['file_name'].values
	width,height=[],[]
	t1=time()
	for ii,fname in enumerate(file_names):
		mod_name =full_data_dir + fname.split('/')[-1]
		image = cv2.imread(mod_name)
		s = image.shape
		#imageHeight = s[0]
		#imageWidth = s[1]
		width.append(s[0])
		height.append(s[1])
		if ii%100==0:
			print('threads %s, index %d, time-cost %f min'%(name,ii,(time()-t1)/60))
		if ii % 1000 == 0:
			joblib.dump([ii,width,height],DATA_DIR+'raw_data/test_size_temp_{}.pkl'.format(name))
	test_file['width']=width
	test_file['height'] = height
	print(name,'test shape',test_file.shape) #153730

	test_file.to_csv(DATA_DIR+'raw_data/test_file_orig_{}.csv'.format(name),index=False)

def get_test_size_multi_thread(thread_num=1):
	test_file = pd.read_csv(DATA_DIR+'test_file.csv')
	test_file['small_width']=test_file['width']
	test_file['small_height'] = test_file['height']
	chunk=math.ceil(len(test_file)/thread_num)
	thread_list=[]
	for ii in range(thread_num):
		sup_file=test_file.iloc[ii*chunk:(ii+1)*chunk]
		thr=threading.Thread(target=get_test_orig_size_split,args=(sup_file,ii))
		thread_list.append(thr)
	for t in thread_list:
		t.setDaemon(True)
		t.start()
	for t in thread_list:
		t.join()

def merge_test_size_file():
	data=pd.DataFrame()
	for name in range(10):
		data_path=DATA_DIR + 'raw_data/test_file_orig_{}.csv'.format(str(name))
		temp=pd.read_csv(data_path)
		data=pd.concat([data,temp])
		print(name,data.shape)

	data.to_csv(DATA_DIR + 'raw_data/test_file.csv',index=False)


<IPython.core.display.Javascript object>

In [5]:
def prepare_data(CCT=True,iNat=True):
	if CCT:
		rewrite_train_data_json('CCT')
	if iNat:
		rewrite_train_data_json('iNat')

	split_train_dev(CCT=CCT,iNat=iNat)
	save_test()

<IPython.core.display.Javascript object>

In [17]:
prepare_data()

len of  data: CCT 196157
len of  data: iNat 25263
use CCT data (196157, 9)
use iNat data (25263, 9)
train shape (221420, 9)
category ratio for train data:
0 130764 0.6074559264162777
1 11115 0.051634032471604766
2 22 0.00010219961442872738
3 8001 0.03716814159292035
4 2224 0.01033145193134044
5 3361 0.015613313822497851
6 8 3.7163496155900866e-05
7 423 0.0019650198592432583
8 8289 0.038506027454532785
9 250 0.001161359254871902
10 2247 0.010438296982788656
11 8578 0.0398485587531647
12 880 0.004087984577149095
13 9668 0.044912085104406196
14 1948 0.00904931131396186
15 36 0.0001672357327015539
16 6128 0.028467238055420063
17 4927 0.022888068195015446
18 3005 0.013959538243560263
19 12766 0.05930364899077881
20 307 0.0014261491649826957
21 142 0.0006596520567672404
22 176 0.000817596915429819
category ratio for dev data:
0 693 0.11259138911454103
1 0 0.0
2 0 0.0
3 14 0.002274573517465475
4 45 0.007311129163281885
5 0 0.0
6 0 0.0
7 0 0.0
8 566 0.09195775792038993
9 0 0.0
10 0 0.0
11 873 

# Detect using faster-RCNN

Từ train_file.csv tạo ở bước prepare data lấy ra những ảnh thuộc dataset nào (CTT/iNat), lấy các thuộc tính: file_name, id, width

.p là một model, `img2det[img_id][iBox=0]` => tức là lấy box của image có id là img_id

Một image khi cropp ở hàm crop_image có thể fail khi:

1. Đọc path không có file đó

2. Image không có trong model

3. Crop xong thì hình rỗng

In [8]:
TRAIN_DATASET={'CCT':'iWildCam_2019_CCT','iNat':'iWildCam_2019_iNat_Idaho','IDFG':'iWildCam_IDFG'} #_images_small
DATA_DIR='./data/'
ANNOTATION_DIR =DATA_DIR+ 'iWildCam_2019_Annotations/'

bbox_detect_dir='data/bbox/'

<IPython.core.display.Javascript object>

In [48]:
def crop_image(img_names,ws,ids,img2det,bbox_detect_dir):
	print('images num:',len(img_names))
	print('detection num:', len(img2det))
	t1=time()
	miss=0
	zero=0
	for ii in range(len(img_names)):
		if ii%5000==0:
			print('processing image',ii,(time()-t1)/(5000-1))
			t1 = time()
		img_file = img_names[ii]
		if os.path.exists(bbox_detect_dir + 'bbox_temp/' + img_file):
			continue
		dirs = img_file.split('/')
		for jj in range(len(dirs)):
			now_dir = '/'.join(dirs[:jj])
			temp_dir = bbox_detect_dir + 'bbox_temp/' + now_dir
			if not os.path.exists(temp_dir):
				os.mkdir(temp_dir)
			crop_dir = bbox_detect_dir + 'cropped_image/' + now_dir
			if not os.path.exists(crop_dir):
				os.mkdir(crop_dir)
		img_id = ids[ii]
		image = cv2.imread(DATA_DIR+img_file)
		if image is None:
			miss+=1
			with open(bbox_detect_dir + 'failed_images.txt','a') as f:
				f.write(img_file+'\n')
# 			cv2.imwrite(bbox_detect_dir + 'cropped_image/' + img_file,np.array([0]))
			continue
		iBox = 0
		try:
			box = img2det[img_id][iBox]
		except KeyError as e:
			miss+=1
			with open(bbox_detect_dir + 'failed_images.txt','a') as f:
				f.write(img_file+'\n')
		#	with open(bbox_detect_dir+'bug_img/img_list.txt','a') as f:
		#		f.write(img_file+'\t'+img_id+'\n')
			continue
            

		imageWidth = image.shape[1]

		ratio = imageWidth / ws[ii]
		box_new = [x * ratio for x in box]
		buffer_scale=0.2
		ww = max(0, int((box_new[3] - box_new[1]) * buffer_scale))
		hh = max(0, int((box_new[2] - box_new[0]) * buffer_scale))

		topRel = int(max(0, box_new[0] - hh))
		leftRel = int(max(0, box_new[1] - ww))
		bottomRel = int(box_new[2] + hh)
		rightRel = int(box_new[3] + ww)

		cropped = image.copy()[leftRel:rightRel, topRel:bottomRel] #.copy()
		if cropped.size == 0:
			miss+=1
			with open(bbox_detect_dir + 'zero_images.txt','a') as f:
				f.write(img_file+'\n')
			cv2.imwrite(bbox_detect_dir + 'cropped_image/' + img_file,np.array([0]))
		else: 
			cv2.imwrite(bbox_detect_dir + 'cropped_image/' + img_file,cropped)


		#img_det = cv2.rectangle(image.copy(),(topRel, leftRel), (bottomRel, rightRel), (0, 255, 0), 3)
		#cv2.imwrite(bbox_detect_dir + 'bbox_temp/' + img_file, img_det)
	print('all miss data',miss)


<IPython.core.display.Javascript object>

In [51]:
train_file=pd.read_csv(DATA_DIR+'train_file.csv')
dev_file = pd.read_csv(DATA_DIR + 'dev_file.csv')
data_file=pd.concat([train_file,dev_file])

<IPython.core.display.Javascript object>

## Detect CTT images

In [11]:
img2det = {}
with open(bbox_detect_dir+'Detection_Results/CCT_Detection_Results_1.p', 'rb') as data_file:
        temp = pickle.load(data_file, encoding='iso-8859-1')
for img, res in zip(temp['images'], temp['detections']):
    img2det[img] = res[:10]
with open(bbox_detect_dir+'Detection_Results/CCT_Detection_Results_2.p', 'rb') as data_file:
    temp = pickle.load(data_file, encoding='iso-8859-1')
for img, res in zip(temp['images'], temp['detections']):
    img2det[img] = res[:10]

<IPython.core.display.Javascript object>

In [19]:
data_cct = data_file[data_file['dataset']=='CCT'].reset_index(drop=True)
img_names = data_cct['file_name'].values
ids = data_cct['id'].values
ws = data_cct['width'].values
crop_image(img_names, ws, ids, img2det, bbox_detect_dir)

images num: 196157
detection num: 196015
processing image 0 7.630920715393079e-10
processing image 5000 0.01300697217919536
processing image 10000 0.01302206542497159
processing image 15000 0.012746658533137901
processing image 20000 0.01308282834812507
processing image 25000 0.013097462307860743
processing image 30000 0.013287499275749315
processing image 35000 0.013158846960279507
processing image 40000 0.01316627723547334
processing image 45000 0.013206293640625097
processing image 50000 0.07300618062569729
processing image 55000 0.012992729637999132
processing image 60000 0.0730627785923243
processing image 65000 0.013204239444461769
processing image 70000 0.07301715980364958
processing image 75000 0.0727845657632503
processing image 80000 0.013417896162774424
processing image 85000 0.07303460871465065
processing image 90000 0.013554148064491438
processing image 95000 0.07255127487670997
processing image 100000 0.05893480217344738
processing image 105000 0.013204147062627857
proces

<IPython.core.display.Javascript object>

## Detect iNat images

In [52]:
img2det = {}
with open(bbox_detect_dir+'Detection_Results/iNat_Idaho_Detection_Results.p', 'rb') as data_file:
    temp = pickle.load(data_file, encoding='iso-8859-1')
for img, res in zip(temp['images'], temp['detections']):
    img2det[img] = res[:10]

<IPython.core.display.Javascript object>

In [ ]:
data_inat = data_file[data_file['dataset']=='iNat'].reset_index(drop=True)
img_names = data_inat['file_name'].values
ids = data_inat['id'].values
ws = data_inat['width'].values

crop_image(img_names, ws, ids, img2det, bbox_detect_dir)

## Detect test images

In [25]:
img2det = {}
with open(bbox_detect_dir + 'Detection_Results/IDFG_Detection_Results_1.p', 'rb') as data_file:
    temp = pickle.load(data_file, encoding='iso-8859-1')
for img, res in zip(temp['images'], temp['detections']):
    img2det[img] = res[:10]
with open(bbox_detect_dir + 'Detection_Results/IDFG_Detection_Results_2.p', 'rb') as data_file:
    temp = pickle.load(data_file, encoding='iso-8859-1')
for img, res in zip(temp['images'], temp['detections']):
    img2det[img] = res[:10]

<IPython.core.display.Javascript object>

In [50]:
data_file=pd.read_csv(DATA_DIR+'test_file.csv')
print('test_file',data_file.shape)

img_names = data_file['file_name'].values
ids = data_file['id'].values
ws = data_file['height'].values

crop_image(img_names, ws, ids, img2det, bbox_detect_dir)

test_file (153730, 9)
images num: 153730
detection num: 153730
processing image 0 8.584785804817213e-10
processing image 5000 0.016586746304338994
processing image 10000 0.01679950028473102
processing image 15000 0.015362671313941133
processing image 20000 0.013955226014151195
processing image 25000 0.01333584616627305
processing image 30000 0.013068846402871273
processing image 35000 0.01268192123570283
processing image 40000 0.012757221444365167
processing image 45000 0.013085560980833251
processing image 50000 0.012729864164361192
processing image 55000 0.011313419719771591
processing image 60000 0.011091036423608574
processing image 65000 0.01085279402338903
processing image 70000 0.011412709921592473
processing image 75000 0.011336010202070932
processing image 80000 0.01114318623116408
processing image 85000 0.010458720114307896
processing image 90000 0.011944347249195322
processing image 95000 0.011972561147742854
processing image 100000 0.011561860177821697
processing image 1050

<IPython.core.display.Javascript object>

# Get cropped successful

Vào folder và lấy *file_name* rùi map với train test csv và xuất ra một train test csv mới trong */data/bbox/cropped-image*

Phần train model cho việc classification thì sẽ lấy trong bbox

In [3]:
def rewrite_cropped_csv():
	def check_file(df,prefix_dir='bbox/cropped_image/',name='train'):
		df=df.reset_index(drop=True)
		print(name,df.shape)
		new_df=pd.DataFrame(data=None,columns=df.columns)
		t1=time()
		file_names=df['file_name'].values
		valid_ind=[]
		new_width,new_height=[],[]
		for ii, file in enumerate(file_names):
			new_path = DATA_DIR + prefix_dir + file
			if os.path.exists(new_path):
				try:
					img = cv2.imread(new_path)
					sh=img.shape
					new_width.append(sh[1])
					new_height.append(sh[0])
					valid_ind.append(ii)
				except:
					continue

		new_df = df.iloc[valid_ind]
		new_df['new_width']=new_width
		new_df['new_height'] = new_height
		print('new_df for:', name, new_df.shape)
		new_df.to_csv(DATA_DIR+prefix_dir+name+'.csv',index=False)
		return new_df
	print('get train cropped csv')
	train = pd.read_csv(DATA_DIR+'train_file.csv')
	new_train=check_file(train,name='train_file')
	print('get dev cropped csv')
	dev = pd.read_csv(DATA_DIR+'dev_file.csv')
	new_dev=check_file(dev,name='dev_file')
	print('get test cropped csv')
	test = pd.read_csv(DATA_DIR+'test_file.csv')
	new_test=check_file(test,name='test_file')

<IPython.core.display.Javascript object>

In [4]:
rewrite_cropped_csv()

get train cropped csv
train_file (215265, 9)
new_df for: train_file (196332, 11)
get dev cropped csv
dev_file (6155, 9)
new_df for: dev_file (6145, 11)
get test cropped csv
test_file (153730, 9)
new_df for: test_file (6041, 11)


<IPython.core.display.Javascript object>

# Get small (200)

Chọn 200 dòng đầu, rùi get những dòng path có ảnh

In [5]:
from shutil import copy2

def rewrite_cropped_csv():
	def check_file(df,prefix_dir='bbox/cropped_image/',name='train'):
		df=df.reset_index(drop=True)
		print(name,df.shape)
		t1=time()
		file_names=df['file_name'].values
		valid_ind=[]
		new_width,new_height=[],[]
		count_successful = 0
		for ii, file in enumerate(file_names):
			new_path = DATA_DIR + prefix_dir + file
			new_copy_path = DATA_DIR + prefix_dir + 'small/' + file
			if os.path.exists(new_path):
				try:
					img = cv2.imread(new_path)
					sh=img.shape
					new_width.append(sh[1])
					new_height.append(sh[0])
					valid_ind.append(ii)
					count_successful += 1
				except:
					continue
				os.makedirs(os.path.dirname(new_copy_path), exist_ok=True)
				copy2(new_path, new_copy_path)
			if(count_successful == 500):
				break
		new_df = df.iloc[valid_ind]
		new_df['new_width']=new_width
		new_df['new_height'] = new_height
		print('new_df for:', name, new_df.shape)
		new_df.to_csv(DATA_DIR+prefix_dir+ 'small/'+name+'.csv',index=False)
		return new_df
# 	print('get train cropped csv')
# 	train = pd.read_csv(DATA_DIR+'train_file.csv')
# 	new_train=check_file(train,name='train_file')
	print('get dev cropped csv')
	dev = pd.read_csv(DATA_DIR+'dev_file.csv')
	new_dev=check_file(dev,name='dev_file')
# 	print('get test cropped csv')
# 	test = pd.read_csv(DATA_DIR+'test_file.csv')
# 	new_test=check_file(test,name='test_file')


<IPython.core.display.Javascript object>

In [6]:
rewrite_cropped_csv()

get dev cropped csv
dev_file (6155, 9)
new_df for: dev_file (500, 11)


<IPython.core.display.Javascript object>

In [37]:
os.path.exists('./data/bbox/cropped_image/test_images/0a9b5c84-2c1c-11e9-bcad-06f10d5896c4.jpg')

True

<IPython.core.display.Javascript object>